In [1]:
###Load latest version of Open Data Canada inventory###
#load_inventory.py
from urllib.request import urlopen
import pandas as pd

#Load inventory from CSV from Open Data Canada
link = input("Copy the current inventory csv file link here.") 

data = pd.read_csv(link)
#Remove French columns that contain "fr" from data
df_names = list(data)

for names in df_names:
    if "fr" in str(names):
        del data[names]
        
#Remove rows that represent datasets that cannot be downloaded
mask = data['portal_url_en'].isnull() 
data_dl = data[-mask]    
#Remove all error nulls ie. "N /A" instead of null
data_dl = data_dl[data_dl.portal_url_en != "N/ A"]

#Removing data sets based on criteria:
# - Canada-wide (not province-specific)
# - No date_published
data_dl = data_dl[data_dl.date_published != "quarterly trimestrielle"]

data_link = input("Copy the data you want to work with as a csv file link here.") 
data_test = pd.read_csv(data_link)

Copy the current inventory csv file link here.http://open.canada.ca/data/dataset/4ed351cf-95d8-4c10-97ac-6b3511f359b7/resource/d0df95a8-31a9-46c9-853b-6952819ec7b4/download/inventory.csv
Copy the data you want to work with as a csv file link here.http://open.canada.ca/data/dataset/4ed351cf-95d8-4c10-97ac-6b3511f359b7/resource/d0df95a8-31a9-46c9-853b-6952819ec7b4/download/inventory.csv


In [12]:
############################## ANALYZING THE INVENTORY ##############################################
#Resort the dataframe based on published date (relevance of data), user_votes (subjective rating about quality), and size (indicator that the data is comprehensive)
data_use = data_dl.sort_values(by=['date_published', 'user_votes', 'size'], ascending=[False, False, False])

#Write csv file
data_use.to_csv("inventory_clean.csv", sep=',', encoding='utf-8')

#Sort data based on number of descriptions that start the same, defined by the first 20 characters of the description. This is an indicator of the functional units that publish data (on the assumption that the same department would describe their data the same way.)
data_use['description_short'] = data_use.description_en.str[0:24]
   
df = data_use.groupby('description_short')['title_en', 'publisher_en'].nunique()
dg = df.sort_values(by=['title_en'], ascending=False)

#Create groups of functional data sets
dg = dg.index.tolist()

In [11]:
dg

['This table is part of a ',
 'Labour force survey esti',
 'This table provides stat',
 'Survey of innovation and',
 'Health behaviour in scho',
 "NSERC's grant and schola",
 'The Road Network File is',
 'Facts and Figures 2015: ',
 'Canadian vehicle survey,',
 'This file contains data ',
 'Survey of innovation, lo',
 'SSHRC’s grant and schola',
 'Canada Mortgage and Hous',
 'Summary table. Find data',
 'Survey of innovation, se',
 'National balance sheet, ',
 'Canadian Internet use su',
 'At the end of each fisca',
 'Survey of household spen',
 'Households and the envir',
 'Socioeconomic overview o',
 'Temporary residents who ',
 'The Atlas of Canada Nati',
 'Aboriginal peoples surve',
 'Survey of Employment, Pa',
 'Capital and repair expen',
 'Gross domestic product (',
 'The Air Pollutant Emissi',
 'International investment',
 'The “Agri-Environmental ',
 'The boundary files portr',
 'Income of immigrants, by',
 'People who have been gra',
 'Registered pension plans',
 'Internationa

In [15]:
#Resort the dataframe based on published date (relevance of data), user_votes (subjective rating about quality), and size (indicator that the data is comprehensive)

def data_prioritize (data_dl):
    data_use = data_dl.sort_values(by=['date_published', 'user_votes', 'size'], ascending=[False, False, False])
    data_use.to_csv("inventory_clean.csv", sep=',', encoding='utf-8')

    #Sort data based on number of descriptions that start the same, defined by the first 20 characters of the description. This is an indicator of the functional units that publish data (on the assumption that the same department would describe their data the same way.)
    data_use['description_short'] = data_use.description_en.str[0:24]
   
    df = data_use.groupby('description_short')['title_en', 'publisher_en'].nunique()
    dg = df.sort_values(by=['title_en'], ascending=False)

    #Create groups of functional data sets
    clumps = dg.index.tolist()
    return dg, data_use


#Bringing functional datasets together - input is data_use, dg
def clump(data, clumps):
    clump = int(input("What clump do you want?"))
    mask = data.description_short == clumps[clump]
    data = data[mask]
    return data

In [23]:
###Get justice survey data###
#http://www.justice.gc.ca/eng/trans/open-ouvert/rsd-drs/njs2016-snj2016/njs2016_data.csv
clump = clump(data_use, dg)

What clump do you want?The National Justice Survey 2016 focuses on the criminal justice system (CJS) to inform the current criminal justice system review and engage with adults 18+ from across Canada.


ValueError: invalid literal for int() with base 10: 'The National Justice Survey 2016 focuses on the criminal justice system (CJS) to inform the current criminal justice system review and engage with adults 18+ from across Canada.'

In [21]:
data_dl

,ref_number,title_en,description_en,publisher_en,date_published,language,size,eligible_for_release,program_alignment_architecture_en,date_released,portal_url_en,user_votes,owner_org,owner_org_title
2,ODI-2016-00059,Association Members,This resource lists the number of members for ...,Agriculture and Agri-Food Canada,2017-02-01,en+fr,0,Y,"Market Access, Negotiations, Sector Competitiv...",2017-03-07,http://open.canada.ca/data/en/dataset/dee0a606...,2,aafc-aac,Agriculture and Agri-Food Canada | Agriculture...
6,ODI-2016-00066,Canadian Cheese Directory,A comprehensive database dedicated solely to C...,Agriculture and Agri-Food Canada,2015-02-09,en+fr,500,Y,"Market Access, Negotiations, Sector Competitiv...",2016-09-24,http://open.canada.ca/data/en/dataset/3c16cd48...,11,aafc-aac,Agriculture and Agri-Food Canada | Agriculture...
10,ODI-2016-00075,Access to Information (ATI) Summaries,"Lists of all completed ATI requests, by month ...",Agriculture and Agri-Food Canada,NaN,en+fr,10000,Y,Communication Services,NaN,http://open.canada.ca/data/en/dataset/0797e893...,2,aafc-aac,Agriculture and Agri-Food Canada | Agriculture...
38,ODI-2016-00124,AAFC Annual Unit Runoff in Canada - 2013,The “AAFC Annual Unit Runoff in Canada - 2013”...,Agriculture and Agri-Food Canada,2013-05-29,en+fr,0,Y,"Science, Innovation, Adoption and Sustainability",2016-09-25,http://open.canada.ca/data/en/dataset/a905bafc...,3,aafc-aac,Agriculture and Agri-Food Canada | Agriculture...
39,ODI-2016-00125,AAFC Infrastructure Flood Mapping in Saskatch...,LiDAR data collected in the fall of 2011 and s...,Agriculture and Agri-Food Canada,2014-04-24,en+fr,0,Y,"Science, Innovation, Adoption and Sustainability",2016-09-24,http://open.canada.ca/data/en/dataset/7ca9fbd2...,2,aafc-aac,Agriculture and Agri-Food Canada | Agriculture...
40,ODI-2016-00134,Agri-Environmental Indicator (AEI) - Proporti...,The proportion of agricultural land as percent...,Agriculture and Agri-Food Canada,2016-07-11,en+fr,0,Y,"Science, Innovation, Adoption and Sustainability",2016-09-26,http://open.canada.ca/data/en/dataset/f95e798a...,2,aafc-aac,Agriculture and Agri-Food Canada | Agriculture...
41,ODI-2016-00136,Agri-Environmental Indicator (AEI) - Agricultu...,The “Agri-Environmental Indicator (AEI) - Agri...,Agriculture and Agri-Food Canada,2016-07-11,en+fr,0,Y,"Science, Innovation, Adoption and Sustainability",2016-09-24,http://open.canada.ca/data/en/dataset/5c9800f3...,1,aafc-aac,Agriculture and Agri-Food Canada | Agriculture...
42,ODI-2016-00141,Agri-Environmental Indicator (AEI) - Livestock...,The “Agri-Environmental Indicator (AEI) - Live...,Agriculture and Agri-Food Canada,2016-07-11,en+fr,0,Y,"Science, Innovation, Adoption and Sustainability",2016-09-25,http://open.canada.ca/data/en/dataset/96271546...,2,aafc-aac,Agriculture and Agri-Food Canada | Agriculture...
43,ODI-2016-00147,Agri-Environmental Indicator (AEI) - Risk of W...,The “Agri-Environmental Indicator (AEI) - Ris...,Agriculture and Agri-Food Canada,2016-07-11,en+fr,0,Y,"Science, Innovation, Adoption and Sustainability",2016-09-24,http://open.canada.ca/data/en/dataset/1f8586b3...,2,aafc-aac,Agriculture and Agri-Food Canada | Agriculture...
44,ODI-2016-00151,Agri-Environmental Indicator (AEI) - Risk of W...,The “Agri-Environmental Indicator (AEI) - Risk...,Agriculture and Agri-Food Canada,2016-07-11,en+fr,0,Y,"Science, Innovation, Adoption and Sustainability",2016-09-24,http://open.canada.ca/data/en/dataset/28baefcf...,10,aafc-aac,Agriculture and Agri-Food Canada | Agriculture...


In [14]:
###Look at the prioritized data sets - prioriti
data_use

,ref_number,title_en,description_en,publisher_en,date_published,language,size,eligible_for_release,program_alignment_architecture_en,date_released,portal_url_en,user_votes,owner_org,owner_org_title,description_short
2187,OD-2016-00070,2016 National Justice Survey,The National Justice Survey 2016 focuses on th...,Department of Justice Canada,2017-10-05,en+fr,4,Y,Stewardship of the Canadian Legal Framework,2017-10-05,http://www.justice.gc.ca/eng/trans/open-ouvert...,0,jus,Department of Justice | Ministère de la Justice,The National Justice Sur
1013,OD-2017-00002,Participation by Province,Participation of Anglophones and Francophones ...,Canadian Security Intelligence Service,2017-09-29,Bilingual (English and French),5,Y,Official Languages Program,2017-10-11,https://csis.gc.ca/dataset/Participation-by-Pr...,0,csis-scrs,Canadian Security Intelligence Service | Servi...,Participation of Angloph
1014,OD-2017-00003,Internal Services to Bilingual Regions,Percentage of resources providing internal ser...,Canadian Security Intelligence Service,2017-09-29,Bilingual (English and French),3,Y,Official Languages Program,2017-10-11,https://csis.gc.ca/dataset/Internal-Services-t...,0,csis-scrs,Canadian Security Intelligence Service | Servi...,Percentage of resources
1012,OD-2017-00001,Paticipation by Employment Category,Participation of Anglophones and Francophones ...,Canadian Security Intelligence Service,2017-09-29,Bilingual (English and French),2,Y,Official Languages Program,2017-10-11,https://csis.gc.ca/dataset/Participation-by-Em...,0,csis-scrs,Canadian Security Intelligence Service | Servi...,Participation of Angloph
1966,OD-2016-00151,Disclosure of Position Reclassifications,These reports provide information on the recla...,Federal Economic Development Agency for Southe...,2017-09-19,en+fr,0,Y,Internal Services,2017-09-19,http://open.canada.ca/en/search/reclassificati...,2,feddevontario,Federal Economic Development Agency for Southe...,These reports provide in
2881,OD-2016-00157,Education Costs as per the Public Accounts of ...,"At the end of each fiscal year, government-wid...",Public Services and Procurement Canada,2017-09-19,en+fr,0,Y,Program 1.3: Receiver General for Canada,2017-09-19,http://open.canada.ca/data/en/dataset/f2f51625...,1,pwgsc-tpsgc,Public Services and Procurement Canada | Servi...,At the end of each fisca
369,CAS-2017-0002,2017-2018 Proactive Disclosure - Contracts ove...,CAS proactive disclosure of contracts over $10...,Courts Administration Service,2017-08-11,en+fr,714,Y,Internal Services - Financial Management Servi...,2017-08-11,http://open.canada.ca/en/search/contracts?f%5B...,0,cas-satj,Courts Administration Service | Service admini...,CAS proactive disclosure
1935,ODI-2017-00001,National Service Provider List,The data provides annual capacity statistics f...,Employment and Social Development Canada,2017-08-09,en+fr,0,Y,4.1 Income Security,2017-09-26,http://open.canada.ca/data/en/dataset/7e0189e3...,0,esdc-edsc,Employment and Social Development Canada | Emp...,The data provides annual
2650,OD-2017-00001,National Research Council peer reviewed public...,Citation information for peer reviewed scienti...,National Research Council Canada,2017-07-31,en+fr,36612882,Y,Science Infrastructure and Management / Nation...,2017-07-31,http://open.canada.ca/data/en/dataset/bcad04f2...,0,nrc-cnrc,National Research Council Canada | Conseil nat...,Citation information for
2833,OD-2016-00106,Summary combined financial statements of enter...,"At the end of each fiscal year, the Receiver g...",Public Services and Procurement Canada,2017-07-27,en+fr,0,Y,Program 1.3: Receiver General for Canada,2017-07-27,http://open.canada.ca/data/en/dataset/02702fc2...,2,pwgsc-tpsgc,Public Services and Procurement Canada | Servi...,At the end of each fisca


In [20]:
data_use["portal_url_en"]

2187    http://www.justice.gc.ca/eng/trans/open-ouvert...
1013    https://csis.gc.ca/dataset/Participation-by-Pr...
1014    https://csis.gc.ca/dataset/Internal-Services-t...
1012    https://csis.gc.ca/dataset/Participation-by-Em...
1966    http://open.canada.ca/en/search/reclassificati...
2881    http://open.canada.ca/data/en/dataset/f2f51625...
369     http://open.canada.ca/en/search/contracts?f%5B...
1935    http://open.canada.ca/data/en/dataset/7e0189e3...
2650    http://open.canada.ca/data/en/dataset/bcad04f2...
2833    http://open.canada.ca/data/en/dataset/02702fc2...
2901    http://open.canada.ca/data/en/dataset/a0b27ff0...
2982    http://open.canada.ca/data/en/dataset/d02f1c50...
808     http://open.cnada.ca/data/en/dataset/d558ea21-...
809     http://open.canada.ca/data/en/dataset/4732b652...
789     http://open.canada.ca/data/en/dataset/c487eab7...
803     http://open.canada.ca/data/en/dataset/59c45960...
9937    http://open.canada.ca/data/en/dataset/ff782362...
793     http:/

In [ ]:
data_test = pd.read_csv(data_link)

In [24]:
justice = pd.read_csv("http://www.justice.gc.ca/eng/trans/open-ouvert/rsd-drs/njs2016-snj2016/njs2016_data.csv", encoding='latin-1')

In [25]:
justice

,SUR_ID,SUR_CPO,SUR_PANEL,DEMO_S1AGE1,QCOMM,QRECRUIT,DEMO_S1PT,DEMO_S1GND,DEMO_S1AGE,S1Q1,...,S2Q28_4,S2Q28_5,S2Q28_6,S2Q28_7,S2Q28_8,S2Q28_9,S2Q28_10,S2Q28_11,S1Q1A,S1Q2A
0,CO1,9.0,1.0,1973.0,1.0,NaN,6.0,2.0,3.0,4.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,99
1,CO10,9.0,1.0,1949.0,NaN,1.0,5.0,1.0,6.0,3.0,...,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,2
2,CO100,9.0,1.0,1988.0,NaN,NaN,5.0,1.0,2.0,4.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,1
3,CO1000,2.0,1.0,1981.0,NaN,NaN,5.0,2.0,3.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,4
4,CO1001,2.0,1.0,1943.0,NaN,1.0,5.0,2.0,6.0,6.0,...,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,3.0,4
5,CO1002,2.0,1.0,1962.0,NaN,1.0,5.0,1.0,4.0,6.0,...,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,3.0,99
6,CO1003,2.0,2.0,1962.0,NaN,1.0,5.0,2.0,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2
7,CO1004,9.0,1.0,1968.0,NaN,1.0,5.0,2.0,4.0,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2
8,CO1005,2.0,1.0,1952.0,NaN,1.0,5.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1
9,CO1006,2.0,1.0,1952.0,NaN,NaN,5.0,1.0,5.0,3.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1
